# EV charger optimal placement: data prep, modeling, optimization, and a Python Streamlit frontend

This notebook walks through the full workflow: data ingestion, cleaning, feature engineering, modeling (regression + classification), clustering- and ILP-based placement optimization, and exporting artifacts for a Streamlit frontend.


## 1) Project To-Do Checklist (tracked in JSON)
We'll maintain a simple tasks.json to track progress across key milestones.
- data prep
- modeling
- placement (KMeans/ILP)
- app (Streamlit)

The next cell creates/loads `tasks.json` and helper functions to toggle tasks.

In [4]:
# Summary: Initialize a simple tasks.json checklist and helper functions; then print current status.
import json, os, pprint
from datetime import datetime

ROOT = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))
TASKS_PATH = os.path.join(ROOT, 'tasks.json')

def init_tasks():
    default = {
        'data_prep': False,
        'modeling': False,
        'placement': False,
        'app': False,
        'updated_at': None,
    }
    if not os.path.exists(TASKS_PATH):
        with open(TASKS_PATH, 'w') as f:
            json.dump(default, f, indent=2)
    return default


def read_tasks():
    if not os.path.exists(TASKS_PATH):
        return init_tasks()
    with open(TASKS_PATH, 'r') as f:
        return json.load(f)


def write_tasks(tasks: dict):
    tasks['updated_at'] = datetime.utcnow().isoformat()
    with open(TASKS_PATH, 'w') as f:
        json.dump(tasks, f, indent=2)


def set_task(name: str, value: bool):
    tasks = read_tasks()
    if name in tasks:
        tasks[name] = value
        write_tasks(tasks)
    else:
        print(f'Unknown task: {name}')


def show_tasks():
    pprint.pprint(read_tasks())

init_tasks()
show_tasks()


{'app': False,
 'data_prep': False,
 'modeling': False,
 'nearest_search': False,
 'placement': False,
 'updated_at': None}


## 2) Environment Setup and Project Scaffolding
Create folders and write a requirements.txt if missing.

In [5]:
# Summary: Create project folders and a minimal requirements.txt if missing.
import os, textwrap, pathlib

folders = [
    'data/raw', 'data/processed', 'models', 'artifacts', 'notebooks', 'src', 'app', 'tests'
]
for d in folders:
    os.makedirs(d, exist_ok=True)

req = pathlib.Path('requirements.txt')
if not req.exists():
    req.write_text(textwrap.dedent('''\
    pandas>=2.2
    numpy>=1.26
    scikit-learn>=1.5
    scikit-learn-extra>=0.3
    scipy>=1.11
    xgboost>=2.0
    lightgbm>=4.3
    imbalanced-learn>=0.12
    geopandas>=0.14
    shapely>=2.0
    folium>=0.15
    geopy>=2.4
    pulp>=2.8
    seaborn>=0.13
    matplotlib>=3.8
    joblib>=1.4
    streamlit>=1.36
    pyarrow>=16.0
    '''))

print('Scaffold ready. You may need to pip install -r requirements.txt in a terminal.')

Scaffold ready. You may need to pip install -r requirements.txt in a terminal.


## 3) Data Ingestion (Kaggle/local) and Raw Snapshot
We will read the large CSV from repo root if present, else from data/raw, and snapshot to Parquet for faster reads.

In [6]:
# Summary: Read EV CSV in chunks, sample ~1M rows, and persist a fast Parquet snapshot.
import pandas as pd
import os
from pathlib import Path

# Resolve project root (parent of notebooks directory)
ROOT_DIR = Path(os.getcwd()).parent

CSV_CANDIDATES = [
    ROOT_DIR / 'Electric_Vehicle_Population_Data.csv',
    ROOT_DIR / 'data' / 'raw' / 'Electric_Vehicle_Population_Data.csv'
]

csv_path = None
for p in CSV_CANDIDATES:
    if p.exists():
        csv_path = p
        break

if csv_path is None:
    raise FileNotFoundError(f'Place Electric_Vehicle_Population_Data.csv at {ROOT_DIR} or {ROOT_DIR / "data" / "raw"}.')

print('Reading CSV (first ~1e6 rows if huge) ...')
# Read in chunks to avoid memory limits
chunks = pd.read_csv(csv_path, chunksize=200_000, low_memory=False)
first = next(chunks)
cols = list(first.columns)

# Take a few chunks to build a profiling sample (adjust N if needed)
sample_list = [first]
for i, c in enumerate(chunks, start=1):
    sample_list.append(c)
    if i >= 4:  # ~1M rows total (5 chunks * 200k)
        break

df_raw = pd.concat(sample_list, ignore_index=True)
print('Raw sample shape:', df_raw.shape)

raw_snapshot = ROOT_DIR / 'data' / 'processed' / 'raw_snapshot.parquet'
raw_snapshot.parent.mkdir(parents=True, exist_ok=True)
df_raw.to_parquet(raw_snapshot, index=False)
print('Saved snapshot to', raw_snapshot)

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

## 4) Data Profiling and Schema Fixes
Peek at columns, types, uniques, and standardize column names to snake_case.

In [ ]:
# Summary: Standardize column names to snake_case, print info and head for quick schema profiling.
import re

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 180)

def to_snake(s: str) -> str:
    s = re.sub(r'[^0-9A-Za-z]+', '_', s)
    s = re.sub(r'_+', '_', s).strip('_')
    return s.lower()

orig_cols = df_raw.columns.tolist()
df = df_raw.copy()
df.columns = [to_snake(c) for c in df.columns]

print('Original -> New column names (first 15):')
for o, n in zip(orig_cols[:15], df.columns[:15]):
    print(f'- {o} -> {n}')

print('\nInfo:')
print(df.info())

print('\nHead:')
df.head()

Original -> New column names (first 15):
- VIN (1-10) -> vin_1_10
- County -> county
- City -> city
- State -> state
- Postal Code -> postal_code
- Model Year -> model_year
- Make -> make
- Model -> model
- Electric Vehicle Type -> electric_vehicle_type
- Clean Alternative Fuel Vehicle (CAFV) Eligibility -> clean_alternative_fuel_vehicle_cafv_eligibility
- Electric Range -> electric_range
- Base MSRP -> base_msrp
- Legislative District -> legislative_district
- DOL Vehicle ID -> dol_vehicle_id
- Vehicle Location -> vehicle_location

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250659 entries, 0 to 250658
Data columns (total 17 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   vin_1_10                                         250659 non-null  object 
 1   county                                           250653 non-null  object 
 2   city                   

,vin_1_10,county,city,state,postal_code,model_year,make,model,electric_vehicle_type,clean_alternative_fuel_vehicle_cafv_eligibility,electric_range,base_msrp,legislative_district,dol_vehicle_id,vehicle_location,electric_utility,2020_census_tract
0,5YJSA1E65N,Yakima,Granger,WA,98932.0,2022,TESLA,MODEL S,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,15.0,187279214,POINT (-120.1871 46.33949),PACIFICORP,5.307700e+10
1,KNDC3DLC5N,Yakima,Yakima,WA,98902.0,2022,KIA,EV6,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,15.0,210098241,POINT (-120.52041 46.59751),PACIFICORP,5.307700e+10
2,5YJYGDEEXL,Snohomish,Everett,WA,98208.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291.0,0.0,44.0,121781950,POINT (-122.18637 47.89251),PUGET SOUND ENERGY INC,5.306104e+10
3,3C3CFFGE1G,Yakima,Yakima,WA,98908.0,2016,FIAT,500,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,14.0,180778377,POINT (-120.60199 46.59817),PACIFICORP,5.307700e+10
4,KNDCC3LD5K,Kitsap,Bremerton,WA,98312.0,2019,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26.0,0.0,26.0,2581225,POINT (-122.65223 47.57192),PUGET SOUND ENERGY INC,5.303508e+10


## 5) Missing Value Handling
Impute numeric with median and categoricals with mode ('Unknown'). Persist the imputers.

In [ ]:
# Summary: Impute missing numeric (median) and categorical (most frequent) values and persist imputers.
from sklearn.impute import SimpleImputer
import numpy as np
import joblib
from pathlib import Path

num_cols = [c for c in df.select_dtypes(include=[np.number]).columns]
cat_cols = [c for c in df.select_dtypes(exclude=[np.number]).columns]

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

if num_cols:
    df[num_cols] = num_imputer.fit_transform(df[num_cols])
if cat_cols:
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

Path('artifacts').mkdir(exist_ok=True)
joblib.dump(num_imputer, 'artifacts/num_imputer.pkl')
joblib.dump(cat_imputer, 'artifacts/cat_imputer.pkl')

print('Imputation complete.')

Imputation complete.


## 6) Irrelevant Column Removal
Drop identifiers and columns with too many missing values.

In [ ]:
# Summary: Drop ID-like columns and those with excessive missing values; keep the rest.
missing_ratio = df.isna().mean().sort_values(ascending=False)

id_like = [c for c in df.columns if any(k in c for k in ['vin', 'vehicle_id', 'id'])]

threshold = 0.8
drop_cols = [c for c, r in missing_ratio.items() if r > threshold]
keep_cols = set(df.columns) - set(id_like) - set(drop_cols)

df = df[list(keep_cols)].copy()
print('Dropped columns:', id_like + drop_cols)
print('Remaining cols:', len(df.columns))

Dropped columns: ['vin_1_10', 'dol_vehicle_id']
Remaining cols: 15


## 7) Text Cleaning
Normalize spaces/case and unify common synonyms (e.g., BEV vs Battery Electric).

In [ ]:
# Summary: Normalize text fields and harmonize EV type labels (BEV/PHEV).
for c in df.select_dtypes(include='object').columns:
    df[c] = df[c].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

if 'electric_vehicle_type' in df.columns:
    df['electric_vehicle_type'] = df['electric_vehicle_type'].str.lower().replace({
        'battery electric vehicle (bev)': 'bev',
        'bev': 'bev',
        'plug-in hybrid electric vehicle (phev)': 'phev',
        'phev': 'phev'
    })

print('Text normalization complete.')

Text normalization complete.


## 8) Feature Transformation (dates, geospatial, numeric casts)
Parse dates/years, extract lat/lon, coerce numeric types where needed.

In [ ]:
# Summary: Coerce likely numeric fields and lat/lon columns to numeric types; parse year if present.
import numpy as np

# Example likely columns (adjust if present)
possible_year_cols = [c for c in df.columns if 'model_year' in c or 'modelyear' in c]
if possible_year_cols:
    yr = possible_year_cols[0]
    df[yr] = pd.to_numeric(df[yr], errors='coerce')

for c in ['latitude', 'longitude', 'lat', 'lon']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Coerce numerics that look numeric but are object
for c in df.columns:
    if df[c].dtype == object:
        sample = df[c].head(100).str.replace(',', '', regex=False)
        if sample.str.match(r'^-?\d+(\.\d+)?$').mean() > 0.7:
            df[c] = pd.to_numeric(df[c].str.replace(',', '', regex=False), errors='ignore')

print('Basic type coercion done.')

Basic type coercion done.


## 9) Outlier Detection and Treatment
Use IQR on key numeric features and clip tails.

In [ ]:
# Summary: Clip numeric outliers using the IQR (1.5x) rule.
def iqr_clip(series: pd.Series, k: float = 1.5):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return series.clip(lower=lo, upper=hi)

for c in df.select_dtypes(include=[np.number]).columns:
    df[c] = iqr_clip(df[c])

print('Outliers clipped via IQR rule.')

Outliers clipped via IQR rule.


## 10) Categorical Encoding and 11) Scaling
Fit OneHotEncoder for categoricals and StandardScaler for numerics; persist.

In [ ]:
# Summary: Fit OneHotEncoder for categoricals and StandardScaler for numerics; save both.
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_cols = [c for c in df.select_dtypes(exclude=[np.number]).columns]
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns]

# scikit-learn >= 1.2 uses sparse_output instead of sparse
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
scaler = StandardScaler()

if cat_cols:
    enc.fit(df[cat_cols])
    joblib.dump(enc, 'artifacts/encoder.pkl')
if num_cols:
    scaler.fit(df[num_cols])
    joblib.dump(scaler, 'artifacts/scaler.pkl')

print('Encoder/scaler persisted.')

Encoder/scaler persisted.


## 12) Aggregation / Grouping (city/county level demand)
Aggregate by city/county to compute EV counts and type splits.

In [ ]:
# Summary: Aggregate by area (city/county/state) to compute EV counts and type splits.
group_keys = [k for k in ['city', 'county', 'state'] if k in df.columns]
if not group_keys:
    # fallback: census_tract or zip
    for k in ['census_tract', 'postal_code', 'zip_code']:
        if k in df.columns:
            group_keys = [k]
            break

print('Grouping by:', group_keys)

def is_type(val, key):
    return 1 if str(val).lower() == key else 0

agg_rows = []
for key, g in df.groupby(group_keys):
    row = {}
    if isinstance(key, tuple):
        for k, v in zip(group_keys, key):
            row[k] = v
    else:
        row[group_keys[0]] = key
    row['ev_count'] = len(g)
    if 'electric_vehicle_type' in g.columns:
        row['bev_count'] = g['electric_vehicle_type'].apply(lambda v: is_type(v, 'bev')).sum()
        row['phev_count'] = g['electric_vehicle_type'].apply(lambda v: is_type(v, 'phev')).sum()
    agg_rows.append(row)

df_agg = pd.DataFrame(agg_rows)
print('Aggregated shape:', df_agg.shape)

df_agg.to_parquet('data/processed/agg_city_county.parquet', index=False)
print('Saved aggregated table.')

Grouping by: ['city', 'county', 'state']
Aggregated shape: (898, 6)
Saved aggregated table.


## 13) Feature Engineering (density, ratios, geo buckets)
Compute simple demand features and centroid lat/lon per area.

In [ ]:
# Summary: Add centroid latitude/longitude and simple ratio features for each area.
# Compute centroid lat/lon if available
lat_col = next((c for c in ['latitude','lat'] if c in df.columns), None)
lon_col = next((c for c in ['longitude','lon'] if c in df.columns), None)

if lat_col and lon_col:
    cent = df.groupby(group_keys)[[lat_col, lon_col]].mean().reset_index().rename(columns={lat_col:'centroid_lat', lon_col:'centroid_lon'})
    df_agg = df_agg.merge(cent, on=group_keys, how='left')

# Simple density proxies
if {'bev_count','phev_count','ev_count'} <= set(df_agg.columns):
    df_agg['bev_ratio'] = (df_agg['bev_count'] / df_agg['ev_count'].replace(0,1)).clip(0,1)
    df_agg['phev_ratio'] = (df_agg['phev_count'] / df_agg['ev_count'].replace(0,1)).clip(0,1)

print('Feature engineering complete.')

df_agg.to_parquet('data/processed/agg_city_county.parquet', index=False)
print('Re-saved aggregated table with features.')

Feature engineering complete.
Re-saved aggregated table with features.


## 14) Class/Target Definition
Create regression target (required chargers) and classification target (high demand).

In [ ]:
# Summary: Create targets—required chargers (regression) and high_demand flag (classification).
import math

vehicles_per_port = 20  # heuristic; adjust as needed
df_agg['required_chargers'] = df_agg['ev_count'].apply(lambda v: int(math.ceil(v / max(1, vehicles_per_port))))

# high demand if above 75th percentile
th = df_agg['ev_count'].quantile(0.75)
df_agg['high_demand'] = (df_agg['ev_count'] >= th).astype(int)

print('Targets created: required_chargers, high_demand (threshold=', th, ')')

df_agg.to_parquet('data/processed/agg_targets.parquet', index=False)

Targets created: required_chargers, high_demand (threshold= 45.25 )


## 15) Train/Validation Split
Split with stratification on high_demand to stabilize classification task.

In [ ]:
# Summary: Split data into train/validation sets (stratified by high_demand) using numeric features.
from sklearn.model_selection import train_test_split

feature_cols = [c for c in df_agg.columns if c not in set(group_keys + ['required_chargers','high_demand'])]
X = df_agg[feature_cols].copy()
y_reg = df_agg['required_chargers']
y_cls = df_agg['high_demand']

# Simple numeric-only subset for baseline models; one-hot could be added later
X_num = X.select_dtypes(include=[np.number]).fillna(0)

X_train, X_val, y_reg_train, y_reg_val, y_cls_train, y_cls_val = train_test_split(
    X_num, y_reg, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)

print('Train/val shapes:', X_train.shape, X_val.shape)

Train/val shapes: (718, 5) (180, 5)


## 16) Baseline Models (Linear/Logistic)
Train and evaluate simple baseline models.

In [ ]:
# Summary: Train/evaluate baseline linear (regression) and logistic (classification) models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score, roc_auc_score
import numpy as np

lin = LinearRegression().fit(X_train, y_reg_train)
reg_pred = lin.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_reg_val, reg_pred))
mae = mean_absolute_error(y_reg_val, reg_pred)

log = LogisticRegression(max_iter=1000).fit(X_train, y_cls_train)
cls_proba = log.predict_proba(X_val)[:,1]
cls_pred = (cls_proba >= 0.5).astype(int)
f1 = f1_score(y_cls_val, cls_pred)
try:
    auc = roc_auc_score(y_cls_val, cls_proba)
except Exception:
    auc = float('nan')

print({'reg_rmse': rmse, 'reg_mae': mae, 'cls_f1': f1, 'cls_auc': auc})

{'reg_rmse': np.float64(0.2694131748942958), 'reg_mae': 0.217652930957749, 'cls_f1': 1.0, 'cls_auc': 1.0}


## 17) Tree-Based Models
RandomForest and Gradient Boosting (XGBoost/LightGBM) for both tasks.

In [ ]:
# Summary: Train/evaluate Random Forest models for regression and classification.
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

rf_reg = RandomForestRegressor(n_estimators=300, random_state=42).fit(X_train, y_reg_train)
rf_reg_pred = rf_reg.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_reg_val, rf_reg_pred))
rf_mae = mean_absolute_error(y_reg_val, rf_reg_pred)

rf_cls = RandomForestClassifier(n_estimators=300, random_state=42).fit(X_train, y_cls_train)
rf_cls_proba = rf_cls.predict_proba(X_val)[:,1]
rf_cls_pred = (rf_cls_proba >= 0.5).astype(int)
rf_f1 = f1_score(y_cls_val, rf_cls_pred)
rf_auc = roc_auc_score(y_cls_val, rf_cls_proba)

print({'rf_reg_rmse': rf_rmse, 'rf_reg_mae': rf_mae, 'rf_cls_f1': rf_f1, 'rf_cls_auc': rf_auc})

{'rf_reg_rmse': np.float64(2.9014265222813287), 'rf_reg_mae': 0.42442592592592576, 'rf_cls_f1': 1.0, 'rf_cls_auc': 1.0}


## 18) Hyperparameter Tuning (RandomizedSearch)
Quick randomized search on RF as a demo (can expand as needed).

In [ ]:
# Summary: Run RandomizedSearchCV to tune Random Forest hyperparameters for both tasks.
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [200, 300, 500],
    'max_depth': [None, 8, 12, 20],
    'min_samples_split': [2, 5, 10],
}

rf_reg_cv = RandomizedSearchCV(RandomForestRegressor(random_state=42), param_grid, n_iter=6, cv=3, n_jobs=-1, random_state=42)
rf_reg_cv.fit(X_train, y_reg_train)
print('Best RF reg params:', rf_reg_cv.best_params_)

rf_cls_cv = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_grid, n_iter=6, cv=3, n_jobs=-1, random_state=42)
rf_cls_cv.fit(X_train, y_cls_train)
print('Best RF cls params:', rf_cls_cv.best_params_)

Best RF reg params: {'n_estimators': 300, 'min_samples_split': 5, 'max_depth': 20}
Best RF cls params: {'n_estimators': 500, 'min_samples_split': 10, 'max_depth': 20}


## 19) Model Evaluation and Selection + 20) Persist Artifacts
Pick the better model(s) and save them with feature list.

In [ ]:
# Summary: Select the best models (CV if available) and persist them plus the feature list.
import json, joblib, os

best_reg = rf_reg_cv.best_estimator_ if 'rf_reg_cv' in globals() else rf_reg
best_cls = rf_cls_cv.best_estimator_ if 'rf_cls_cv' in globals() else rf_cls

os.makedirs('models', exist_ok=True)
joblib.dump(best_reg, 'models/best_regressor.joblib')
joblib.dump(best_cls, 'models/best_classifier.joblib')

with open('artifacts/feature_list.json','w') as f:
    json.dump({'features': feature_cols, 'numeric_only': list(X_num.columns)}, f, indent=2)

print('Saved selected models and feature list.')

Saved selected models and feature list.


## 21) Optimal Station Placement via Clustering (KMeans)
Cluster high-demand centroids to get k candidate station locations.

In [ ]:
# Summary: Cluster high-demand areas to suggest candidate station locations via KMeans.
from sklearn.cluster import KMeans

hd = df_agg[df_agg['high_demand']==1].dropna(subset=['centroid_lat','centroid_lon'])
coords = hd[['centroid_lat','centroid_lon']].to_numpy()

k = min(20, max(2, len(hd)//10))  # heuristic
km = KMeans(n_clusters=k, n_init=10, random_state=42)
labels = km.fit_predict(coords)
centroids = km.cluster_centers_

stations = pd.DataFrame({
    'station_id': [f'kmeans_{i}' for i in range(len(centroids))],
    'latitude': centroids[:,0],
    'longitude': centroids[:,1],
    'method': 'kmeans'
})

stations.to_csv('artifacts/stations.csv', index=False)
print('Saved KMeans station candidates:', stations.shape)

KeyError: ['centroid_lat', 'centroid_lon']

In [ ]:
# Summary: If centroid_lat/lon are missing, derive them from available lat/lon or parse location strings.
# Fallback: derive centroid_lat/lon if missing using available columns
import re
import pandas as pd

if not set(['centroid_lat','centroid_lon']).issubset(df_agg.columns):
    # Try to find lat/lon columns in df
    lat_candidates = [c for c in df.columns if re.search(r'lat', c, re.I)]
    lon_candidates = [c for c in df.columns if re.search(r'lon|lng|long', c, re.I)]
    lat2 = next((c for c in lat_candidates if df[c].notna().any()), None)
    lon2 = next((c for c in lon_candidates if df[c].notna().any()), None)

    merged = False
    if lat2 and lon2:
        cent = (
            df.groupby(group_keys)[[lat2, lon2]]
              .mean()
              .reset_index()
              .rename(columns={lat2:'centroid_lat', lon2:'centroid_lon'})
        )
        df_agg = df_agg.merge(cent, on=group_keys, how='left')
        merged = True

    if not merged:
        # Try to parse a location field like 'Vehicle Location' or 'Location'
        loc_col = next((c for c in df.columns if 'location' in c.lower()), None)
        if loc_col is not None:
            def extract_lat_lon(s):
                if pd.isna(s):
                    return pd.Series([None, None])
                s = str(s)
                # Format: POINT (lon lat)
                m = re.search(r'POINT\s*\(\s*(-?\d+\.\d+)\s+(-?\d+\.\d+)\s*\)', s)
                if m:
                    lon = float(m.group(1)); lat = float(m.group(2))
                    return pd.Series([lat, lon])
                # Format: (lat, lon)
                m = re.search(r'\(\s*(-?\d+\.\d+)\s*,\s*(-?\d+\.\d+)\s*\)', s)
                if m:
                    lat = float(m.group(1)); lon = float(m.group(2))
                    return pd.Series([lat, lon])
                return pd.Series([None, None])

            tmp = df[group_keys + [loc_col]].copy()
            tmp[['__lat','__lon']] = tmp[loc_col].apply(extract_lat_lon)
            cent = (
                tmp.dropna(subset=['__lat','__lon'])
                   .groupby(group_keys)[['__lat','__lon']]
                   .mean()
                   .reset_index()
                   .rename(columns={'__lat':'centroid_lat','__lon':'centroid_lon'})
            )
            df_agg = df_agg.merge(cent, on=group_keys, how='left')

print('df_agg centroid cols present:', [c for c in df_agg.columns if c in ['centroid_lat','centroid_lon']])
print('Non-null centroid rows:', df_agg[['centroid_lat','centroid_lon']].dropna().shape[0])

df_agg centroid cols present: ['centroid_lat', 'centroid_lon']
Non-null centroid rows: 898


## 22) Optimal Station Placement via Facility Location (ILP with PuLP)
Optional: p-median style optimization from demand centroids to candidate stations.

In [ ]:
# Summary: Solve a p-median facility location problem to pick p best station sites among candidates.
from math import radians, sin, cos, asin, sqrt
from itertools import product
import pulp as pl

# Distance helper (haversine km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

# Use KMeans centroids as candidate sites
cand = stations[['station_id','latitude','longitude']].copy()
demand = hd[['centroid_lat','centroid_lon']].copy().reset_index(drop=True)

p = min(10, len(cand))  # number of stations to open

# Build distance matrix
D = {}
for i in range(len(demand)):
    for j in range(len(cand)):
        D[(i,j)] = haversine_km(demand.loc[i,'centroid_lat'], demand.loc[i,'centroid_lon'], cand.loc[j,'latitude'], cand.loc[j,'longitude'])

prob = pl.LpProblem('p_median', pl.LpMinimize)
open_j = pl.LpVariable.dicts('open', list(range(len(cand))), 0, 1, pl.LpBinary)
assign_ij = pl.LpVariable.dicts('assign', (list(range(len(demand))), list(range(len(cand)))), 0, 1, pl.LpBinary)

# Objective: minimize distance
prob += pl.lpSum(D[(i,j)] * assign_ij[i][j] for i in range(len(demand)) for j in range(len(cand)))

# Each demand assigned to one open site
for i in range(len(demand)):
    prob += pl.lpSum(assign_ij[i][j] for j in range(len(cand))) == 1
    for j in range(len(cand)):
        prob += assign_ij[i][j] <= open_j[j]

# Exactly p sites open
prob += pl.lpSum(open_j[j] for j in range(len(cand))) == p

_ = prob.solve(pl.PULP_CBC_CMD(msg=False))
chosen = [j for j in range(len(cand)) if open_j[j].value() == 1]

ilp_sites = cand.iloc[chosen].copy()
ilp_sites['method'] = 'ilp_p_median'

all_sites = pd.concat([stations, ilp_sites], ignore_index=True)
all_sites.to_csv('artifacts/stations.csv', index=False)
print('ILP chose', len(ilp_sites), 'sites. Updated artifacts/stations.csv')

ILP chose 10 sites. Updated artifacts/stations.csv


In [ ]:
# Summary: Plot suggested station sites on a Folium map and save to artifacts.
import folium

if not stations_df.empty:
    m = folium.Map(location=[stations_df['latitude'].mean(), stations_df['longitude'].mean()], zoom_start=6)
    for _, r in stations_df.iterrows():
        folium.Marker([r['latitude'], r['longitude']], popup=r['station_id']).add_to(m)
    m.save('artifacts/stations_map.html')
    print('Saved artifacts/stations_map.html')
else:
    print('No stations to plot yet.')

Saved artifacts/stations_map.html


## 26) Export GeoJSON/CSV for App
Export artifacts for the Streamlit app to load quickly.

In [ ]:
# Summary: Export aggregated demand and station recommendations as CSVs for the Streamlit app.
df_agg.to_csv('artifacts/aggregated_demand.csv', index=False)
stations_df.to_csv('artifacts/stations.csv', index=False)
print('Exported artifacts for app.')

Exported artifacts for app.


## 27) Streamlit App: Nearest Charger and Planner Pages
We created `app/app.py`. After running this notebook to generate artifacts, launch the app to test.

## 28) Packaging and Run Scripts (CLI + Streamlit)
We added `src/cli.py` to guide steps. To run the app after artifacts generation, use a terminal:
- streamlit run app/app.py

## 29) Unit Tests for Core Utils
You can add pytest tests under `tests/` as needed (omitted here for brevity).